In [1]:
def align_affine_gap(seq1, seq2, score_matrix, gap_open, gap_extend):
    n, m = len(seq1), len(seq2)

    # M: score of alignment ending with a match/mismatch
    # I: score of alignment ending with a gap in seq1
    # D: score of alignment ending with a gap in seq2
    M = [[-float('inf')] * (m + 1) for _ in range(n + 1)]
    I = [[-float('inf')] * (m + 1) for _ in range(n + 1)]
    D = [[-float('inf')] * (m + 1) for _ in range(n + 1)]

    M[0][0] = 0
    for i in range(1, n + 1):
        D[i][0] = -gap_open - gap_extend * (i - 1)
    for j in range(1, m + 1):
        I[0][j] = -gap_open - gap_extend * (j - 1)

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match_score = score_matrix.get((seq1[i-1], seq2[j-1]), score_matrix.get((seq2[j-1], seq1[i-1])))
            M[i][j] = match_score + max(M[i-1][j-1], I[i-1][j-1], D[i-1][j-1])
            
            I[i][j] = max(M[i][j-1] - gap_open, I[i][j-1] - gap_extend, D[i][j-1] - gap_open)
            
            D[i][j] = max(M[i-1][j] - gap_open, I[i-1][j] - gap_open, D[i-1][j] - gap_extend)

    max_score = max(M[n][m], I[n][m], D[n][m])

    # Traceback
    aligned_seq1, aligned_seq2 = [], []
    i, j = n, m

    if M[n][m] == max_score:
        current = 'M'
    elif I[n][m] == max_score:
        current = 'I'
    else:
        current = 'D'

    while i > 0 or j > 0:
        if current == 'M':
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append(seq2[j-1])
            
            prev_score = M[i][j] - score_matrix.get((seq1[i-1], seq2[j-1]), score_matrix.get((seq2[j-1], seq1[i-1])))
            if abs(prev_score - M[i-1][j-1]) < 1e-9:
                current = 'M'
            elif abs(prev_score - I[i-1][j-1]) < 1e-9:
                current = 'I'
            else:
                current = 'D'
            i -= 1
            j -= 1
        elif current == 'I':
            aligned_seq1.append('-')
            aligned_seq2.append(seq2[j-1])
            
            if abs(I[i][j] - (M[i][j-1] - gap_open)) < 1e-9:
                current = 'M'
            elif abs(I[i][j] - (I[i][j-1] - gap_extend)) < 1e-9:
                current = 'I'
            else:
                current = 'D'
            j -= 1
        else:  # current == 'D'
            aligned_seq1.append(seq1[i-1])
            aligned_seq2.append('-')
            
            if abs(D[i][j] - (M[i-1][j] - gap_open)) < 1e-9:
                current = 'M'
            elif abs(D[i][j] - (I[i-1][j] - gap_open)) < 1e-9:
                current = 'I'
            else:
                current = 'D'
            i -= 1

    return int(max_score), ''.join(reversed(aligned_seq1)), ''.join(reversed(aligned_seq2))


def get_blosum62():
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    scores = [
        [ 4,  0, -2, -1, -2,  0, -2, -1, -1, -1, -1, -2, -1, -1, -1,  1,  0,  0, -3, -2],
        [ 0,  9, -3, -4, -2, -3, -3, -1, -3, -1, -1, -3, -3, -3, -3, -1, -1, -1, -2, -2],
        [-2, -3,  6,  2, -3, -1, -1, -3, -1, -4, -3,  1, -1,  0, -2,  0, -1, -3, -4, -3],
        [-1, -4,  2,  5, -3, -2,  0, -3,  1, -3, -2,  0, -1,  2,  0,  0, -1, -2, -3, -2],
        [-2, -2, -3, -3,  6, -3, -1,  0, -3,  0,  0, -3, -4, -3, -3, -2, -2, -1,  1,  3],
        [ 0, -3, -1, -2, -3,  6, -2, -4, -2, -4, -3,  0, -2, -2, -2,  0, -2, -3, -2, -3],
        [-2, -3, -1,  0, -1, -2,  8, -3, -1, -3, -2,  1, -2,  0,  0, -1, -2, -3, -2,  2],
        [-1, -1, -3, -3,  0, -4, -3,  4, -3,  2,  1, -3, -3, -3, -3, -2, -1,  3, -3, -1],
        [-1, -3, -1,  1, -3, -2, -1, -3,  5, -2, -1,  0, -1,  1,  2,  0, -1, -2, -3, -2],
        [-1, -1, -4, -3,  0, -4, -3,  2, -2,  4,  2, -3, -3, -2, -2, -2, -1,  1, -2, -1],
        [-1, -1, -3, -2,  0, -3, -2,  1, -1,  2,  5, -2, -2,  0, -1, -1, -1,  1, -1, -1],
        [-2, -3,  1,  0, -3,  0,  1, -3,  0, -3, -2,  6, -2,  0,  0,  1,  0, -3, -4, -2],
        [-1, -3, -1, -1, -4, -2, -2, -3, -1, -3, -2, -2,  7, -1, -2, -1, -1, -2, -4, -3],
        [-1, -3,  0,  2, -3, -2,  0, -3,  1, -2,  0,  0, -1,  5,  1,  0, -1, -2, -2, -1],
        [-1, -3, -2,  0, -3, -2,  0, -3,  2, -2, -1,  0, -2,  1,  5, -1, -1, -3, -3, -2],
        [ 1, -1,  0,  0, -2,  0, -1, -2,  0, -2, -1,  1, -1,  0, -1,  4,  1, -2, -3, -2],
        [ 0, -1, -1, -1, -2, -2, -2, -1, -1, -1, -1,  0, -1, -1, -1,  1,  5,  0, -2, -2],
        [ 0, -1, -3, -2, -1, -3, -3,  3, -2,  1,  1, -3, -2, -2, -3, -2,  0,  4, -3, -1],
        [-3, -2, -4, -3,  1, -2, -2, -3, -3, -2, -1, -4, -4, -2, -3, -3, -2, -3, 11,  2],
        [-2, -2, -3, -2,  3, -3,  2, -1, -2, -1, -1, -2, -3, -1, -2, -2, -2, -1,  2,  7]
    ]
    
    blosum62 = {}
    for i, aa1 in enumerate(amino_acids):
        for j, aa2 in enumerate(amino_acids):
            blosum62[(aa1, aa2)] = scores[i][j]
    
    return blosum62

# Main execution
# Read sequences from a file or define them here
# Example with file reading:
try:
    with open('sample_inputs/p1/rosalind_ba5j.txt', 'r') as f:
        seq1 = f.readline().strip()
        seq2 = f.readline().strip()
except FileNotFoundError:
    print("Input file not found. Using default sequences.")
    seq1 = "PLEASANTLY"
    seq2 = "MEANLY"


# Using BLOSUM62 with standard penalties
blosum62 = get_blosum62()
gap_open = 11
gap_extend = 1

score, aligned1, aligned2 = align_affine_gap(seq1, seq2, blosum62, gap_open, gap_extend)

print(score)
print(aligned1)
print(aligned2)


236
IAG------QTLCSGDSTSPDLYNGPHSTKLKSNWPQLINNHQPAPSGIRQVWETQF----YNFKQV--VPAGRNETEDT-VMWEFYQHYWCLSTQWLKM
IAAHMVSWQQTLCSGDMTSPDLYNGKISTKLKSNWPFLI---------IRQVWETQLVSSEYAFKQVGLGPAGRNETEWTHWMDQHYMYNICLSTQWLKM
